In [1]:
import os
import codecs

data_directory = os.path.join('dataset')
review_filepath = os.path.join(data_directory,
                                   'reviews2.json')

with codecs.open(review_filepath, encoding='utf_8') as f:
    first_business_record = f.readline() 

print first_business_record

{"type": "review", "business_id": "1CBs84C-a-cuA3vncXVSAw", "votes": {"cool": 0, "useful": 0, "funny": 0}, "review_id": "0a-pCW4guXIlWNpVeBHChg", "text": "The pizza is terrible, but if you need a place to watch a game or just down some pitchers, this place works.\n\nOh, and the pasta is even worse than the pizza.", "stars": 2, "user_id": "90wm_01FAIqhcgV_mPON9Q", "date": "2006-07-26"}



In [1]:
import json

result_directory = os.path.join('result')
review_txt_filepath = os.path.join(result_directory,
                                   'review_text.txt')
review_count = 0
    
with codecs.open(review_txt_filepath, 'w', encoding='utf_8') as review_txt_file:
    with codecs.open(review_filepath, encoding='utf_8') as review_file:
        for review_json in review_file:
            review = json.loads(review_json)
            review_txt_file.write(review[u'text'].replace('\n', '\\n') + '\n')
            review_count += 1

NameError: name 'os' is not defined

In [3]:
import spacy
import pandas as pd
import itertools as it

nlp = spacy.load('en')

In [4]:
with codecs.open(review_txt_filepath, encoding='utf_8') as f:
    sample_review = list(it.islice(f, 8, 9))[0]
    sample_review = sample_review.replace('\\n', '\n')
        
print sample_review

Their margherita pizza is great! Don't get their pastas--they're badly made, I can make way better pasta. There's sports on and a pool table. The prices are good but unfortunately there's no discount for buying in bulk really.



In [5]:
%%time
parsed_review = nlp(sample_review)

CPU times: user 5.75 ms, sys: 0 ns, total: 5.75 ms
Wall time: 4.67 ms


In [6]:
print parsed_review

Their margherita pizza is great! Don't get their pastas--they're badly made, I can make way better pasta. There's sports on and a pool table. The prices are good but unfortunately there's no discount for buying in bulk really.



In [7]:
for num, sentence in enumerate(parsed_review.sents):
    print 'Sentence {}:'.format(num+1)
    print sentence
    print ''

Sentence 1:
Their margherita pizza is great!

Sentence 2:
Don't get their pastas--

Sentence 3:
they're badly made, I can make way better pasta.

Sentence 4:
There's sports on and a pool table.

Sentence 5:
The prices are good but unfortunately there's no discount for buying in bulk really.




In [8]:
token_lemma = [token.lemma_ for token in parsed_review]

In [9]:
token_lemma

[u'their',
 u'margherita',
 u'pizza',
 u'be',
 u'great',
 u'!',
 u'do',
 u'not',
 u'get',
 u'their',
 u'pasta',
 u'--',
 u"they're",
 u'badly',
 u'make',
 u',',
 u'i',
 u'can',
 u'make',
 u'way',
 u'better',
 u'pasta',
 u'.',
 u'there',
 u"'",
 u'sport',
 u'on',
 u'and',
 u'a',
 u'pool',
 u'table',
 u'.',
 u'the',
 u'price',
 u'be',
 u'good',
 u'but',
 u'unfortunately',
 u'there',
 u"'",
 u'no',
 u'discount',
 u'for',
 u'buy',
 u'in',
 u'bulk',
 u'really',
 u'.',
 u'\n']

In [10]:
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

No handlers could be found for logger "gensim.models.word2vec"


In [11]:
def punct_space(token):
    return token.is_punct or token.is_space

def line_review(filename):
    with codecs.open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    for parsed_review in nlp.pipe(line_review(filename), batch_size=10000, n_threads=4):        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [12]:
unigram_sentences_filepath = os.path.join(result_directory,'unigram_sentences_all.txt')

In [13]:
with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
    for sentence in lemmatized_sentence_corpus(review_txt_filepath):
        f.write(sentence + '\n')

In [14]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

In [15]:
for unigram_sentence in it.islice(unigram_sentences, 230, 240):
    print u' '.join(unigram_sentence)
    print u''

i mostly come here to use their change machine

they have about 20+ thing on tap

and if you do not count thing that be not beer e.g. bud they still have about 12+ beer on tap

that make me really happy

the casual pizza/beer atmosphere be wonderful for a late dinner last night

the folk be really nice and the customer be not super crazy either like you would expect sometimes in a college town

also they have thing like pool table and arcade machine which make them pretty awesome in my book

the pizza be really good

we have a special pizza with feta tomato and basil and a basket of fried mushroos and zuccini

the wait be reasonable for a pizza make from scratch



In [16]:
bigram_model_filepath = os.path.join(result_directory, 'bigram_model_all')

In [17]:
%%time

bigram_model = Phrases(unigram_sentences)
bigram_model.save(bigram_model_filepath)

bigram_model = Phrases.load(bigram_model_filepath)

CPU times: user 4.55 s, sys: 72.1 ms, total: 4.63 s
Wall time: 4.61 s


In [18]:
bigram_sentences_filepath = os.path.join(result_directory,'bigram_sentences_all.txt')

In [19]:
with codecs.open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:

    for unigram_sentence in unigram_sentences:

        bigram_sentence = u' '.join(bigram_model[unigram_sentence])

        f.write(bigram_sentence + '\n')

In [20]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [21]:
for bigram_sentence in it.islice(bigram_sentences, 230, 240):
    print u' '.join(bigram_sentence)
    print u''

i mostly come_here to use their change machine

they have about 20+ thing on_tap

and if_you do not count thing that be not beer e.g. bud they still have about 12+ beer on_tap

that make me really happy

the casual pizza/beer atmosphere be wonderful for a late dinner last_night

the folk be really nice and the customer be not super crazy either like you would expect sometimes in a college_town

also they have thing like pool_table and arcade machine which make them pretty awesome in my book

the pizza be really good

we have a special pizza with feta tomato and basil and a basket of fried mushroos and zuccini

the wait be reasonable for a pizza make from_scratch



In [22]:
trigram_model_filepath = os.path.join(result_directory,'trigram_model_all')

In [23]:
trigram_model = Phrases(bigram_sentences)

trigram_model.save(trigram_model_filepath)
    
# load the finished model from disk
trigram_model = Phrases.load(trigram_model_filepath)

In [24]:
trigram_sentences_filepath = os.path.join(result_directory,'trigram_sentences_all.txt')

In [25]:
with codecs.open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:

    for bigram_sentence in bigram_sentences:

        trigram_sentence = u' '.join(trigram_model[bigram_sentence])

        f.write(trigram_sentence + '\n')

In [26]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [27]:
for trigram_sentence in it.islice(trigram_sentences, 230, 240):
    print u' '.join(trigram_sentence)
    print u''

i mostly come_here to use their change machine

they have about 20+ thing on_tap

and if_you do_not count thing that be not beer e.g. bud they still have about 12+ beer_on_tap

that make me really happy

the casual pizza/beer atmosphere be wonderful for a late dinner last_night

the folk be really nice and the customer be not super crazy either like you would expect sometimes in a college_town

also they have thing like pool_table and arcade machine which make them pretty awesome in my_book

the pizza be really good

we have a special pizza with feta tomato and basil and a basket of fried mushroos and zuccini

the wait be reasonable for a pizza make from_scratch



In [28]:
trigram_reviews_filepath = os.path.join(result_directory,'trigram_transformed_reviews_all.txt')

In [29]:
with codecs.open(trigram_reviews_filepath, 'w', encoding='utf_8') as f:

    for parsed_review in nlp.pipe(line_review(review_txt_filepath),
                                  batch_size=10000, n_threads=4):

        # lemmatize the text, removing punctuation and whitespace
        unigram_review = [token.lemma_ for token in parsed_review
                          if not punct_space(token)]

        # apply the first-order and second-order phrase models
        bigram_review = bigram_model[unigram_review]
        trigram_review = trigram_model[bigram_review]

        # remove any remaining stopwords
        trigram_review = [term for term in trigram_review
                          if term not in spacy.en.STOPWORDS]

        # write the transformed review as a line in the new file
        trigram_review = u' '.join(trigram_review)
        f.write(trigram_review + '\n')

In [30]:
print u'Original:' + u'\n'

for review in it.islice(line_review(review_txt_filepath), 11, 12):
    print review

print u'----' + u'\n'
print u'Transformed:' + u'\n'

with codecs.open(trigram_reviews_filepath, encoding='utf_8') as f:
    for review in it.islice(f, 11, 12):
        print review

Original:

This is a nice La Val's. It has big screen TVs playing sports and a pool table in the back. The pizza is just ok as always, but the garlic bread is delicious. 

Back in the days it was nice to take a break from the dreariness of Soda Hall have dinner, and play a game of pool, before going back to program.

----

Transformed:

nice la_val_'s big_screen_tv play_sport pool_table pizza ok garlic_bread delicious day nice break dreariness soda_hall dinner play game pool go_back program



In [31]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings
import cPickle as pickle

In [32]:
trigram_dictionary_filepath = os.path.join(result_directory, 'trigram_dict_all.dict')

In [33]:
trigram_reviews = LineSentence(trigram_reviews_filepath)

# learn the dictionary by iterating over all of the reviews
trigram_dictionary = Dictionary(trigram_reviews)

# filter tokens that are very rare or too common from
# the dictionary (filter_extremes) and reassign integer ids (compactify)
trigram_dictionary.filter_extremes(no_below=10, no_above=0.4)
trigram_dictionary.compactify()

trigram_dictionary.save(trigram_dictionary_filepath)

# load the finished dictionary from disk
trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

In [34]:
trigram_bow_filepath = os.path.join(result_directory,'trigram_bow_corpus_all.mm')

In [35]:
def trigram_bow_generator(filepath):
    for review in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(review)

In [36]:
%%time
# generate bag-of-words representations for
# all reviews and save them as a matrix
MmCorpus.serialize(trigram_bow_filepath,
                   trigram_bow_generator(trigram_reviews_filepath))
    
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus(trigram_bow_filepath)

CPU times: user 4.06 s, sys: 16 ms, total: 4.07 s
Wall time: 4.07 s


In [37]:
lda_model_filepath = os.path.join(result_directory, 'lda_model_all')

In [38]:
# lda
%%time
with warnings.catch_warnings():
    warnings.simplefilter('ignore')

    lda = LdaMulticore(trigram_bow_corpus,
                       num_topics=50,
                       id2word=trigram_dictionary,
                       workers=3)

lda.save(lda_model_filepath)
    
# load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

CPU times: user 50.5 s, sys: 3.54 s, total: 54.1 s
Wall time: 1min 29s


In [39]:
def explore_topic(topic_number, topn=25):
    print u'{:20} {}'.format(u'term', u'frequency') + u'\n'

    for term, frequency in lda.show_topic(topic_number, topn=25):
        print u'{:20} {:.3f}'.format(term, round(frequency, 3))

In [43]:
explore_topic(topic_number=5)

term                 frequency

order                0.012
price                0.011
food                 0.010
do_not               0.010
like                 0.009
great                0.008
nice                 0.007
place                0.007
sandwich             0.007
2                    0.007
this_place           0.007
love                 0.007
'                    0.007
pretty               0.007
service              0.006
chicken              0.006
taste                0.005
breakfast            0.005
home_fries           0.005
eat                  0.005
dish                 0.005
's                   0.005
want                 0.005
if_you               0.004
location             0.004


In [46]:
topic_names = {0: u'mexican',
               1: u'menu',
               2: u'thai',
               3: u'steak',
               4: u'donuts & appetizers',
               5: u'specials',
               6: u'soup',
               7: u'wings, sports bar',
               8: u'foreign language',
               9: u'las vegas',
               10: u'chicken',
               11: u'aria buffet',
               12: u'noodles',
               13: u'ambience & seating',
               14: u'sushi',
               15: u'arizona',
               16: u'family',
               17: u'price',
               18: u'sweet',
               19: u'waiting',
               20: u'general',
               21: u'tapas',
               22: u'dirty',
               23: u'customer service',
               24: u'restrooms',
               25: u'chinese',
               26: u'gluten free',
               27: u'pizza',
               28: u'seafood',
               29: u'amazing',
               30: u'eat, like, know, want',
               31: u'bars',
               32: u'breakfast',
               33: u'location & time',
               34: u'italian',
               35: u'barbecue',
               36: u'arizona',
               37: u'indian',
               38: u'latin & cajun',
               39: u'burger & fries',
               40: u'vegetarian',
               41: u'lunch buffet',
               42: u'customer service',
               43: u'taco, ice cream',
               44: u'high cuisine',
               45: u'healthy',
               46: u'salad & sandwich',
               47: u'greek',
               48: u'poor experience',
               49: u'wine & dine'}

In [47]:
topic_names_filepath = os.path.join(result_directory, 'topic_names.pkl')

with open(topic_names_filepath, 'w') as f:
    pickle.dump(topic_names, f)

In [48]:
LDAvis_data_filepath = os.path.join(result_directory, 'ldavis_prepared')

In [ ]:
LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus,
                                          trigram_dictionary)

with open(LDAvis_data_filepath, 'w') as f:
    pickle.dump(LDAvis_prepared, f)
        
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath) as f:
    LDAvis_prepared = pickle.load(f)

In [ ]:
pyLDAvis.display(LDAvis_prepared)